In [1]:
import random
import networkx as nx


class Individual:
    def __init__(self, sets, graph):

        self.size = sum(len(s) for s in sets)
        self.graph = graph
        self.sorted_edges = list(self.graph.edges())
        # self.sorted_edges = sorted(self.sorted_edges, key=lambda x:(x[0],x[1]))
        self.sorted_edges = [tuple(sorted(t)) for t in self.sorted_edges]
        self.sorted_edges = sorted(self.sorted_edges, key=lambda x: (x[0], x[1]))
        self.gene = {i: (None, None) for i in self.sorted_edges}
        self._populate_individual(sets)

    def _populate_individual(self, sets):
        visited = {t: 0 for t in self.sorted_edges}

        for set in sets:
            random_set = list(set)
            random.shuffle(random_set)
            for cur_edge in set:
                value_assigned = False
                for other_edge in random_set:
                    if visited[other_edge] or other_edge == cur_edge:
                        continue
                    if self.adjacent(cur_edge, other_edge):
                        self.gene[cur_edge] = other_edge
                        visited[other_edge] = 1
                        value_assigned=True
                        break
                if not value_assigned:
                    for other_edge in random_set:
                        if not visited[other_edge]:
                            self.gene[cur_edge] = other_edge
                            visited[other_edge] = 1
                            break

    def adjacent(self, t1, t2):
        if t1[0] == t2[0] or t1[0] == t2[1] or t1[1] == t2[0] or t1[1] == t2[1]:
            return True
        return False

    def decode(self):
        labels = {l: 0 for l in self.sorted_edges}
        c = 1
        for locus, gene_value in self.gene.items():
            if labels[locus] == 0 and labels[gene_value] == 0:
                labels[locus] = c
                labels[gene_value] = c
                c += 1
            elif labels[locus] == 0:
                labels[locus] = labels[gene_value]
            elif labels[gene_value] == 0:
                labels[gene_value] = labels[locus]

        value_to_keys = {}
        for key, value in labels.items():
            if value in value_to_keys:
                value_to_keys[value].add(key)
            else:
                value_to_keys[value] = {key}
        return list(value_to_keys.values())

    def get_gene(self, index):
        return self.gene[self.sorted_edges[index]]

    def set_gene(self, index, gene_value):
        self.gene[self.sorted_edges[index]] = gene_value

    def get_loci(self,index):
        return self.sorted_edges[index]

    def __eq__(self, other):
        for i in range(len(self.gene)):
            if (self.gene[i] != other.gene[i] or self.sorted_edges[i] != other.loci[i]):
                return False
        return True

    def __str__(self):
        loci_str = ', '.join(map(str, self.gene.keys()))
        gene_str = ', '.join(map(str, self.gene.values()))

        return f"Loci: [{loci_str}]\nGene: [{gene_str}]"




In [2]:
def most_frequent_label(labels, graph, v):
    neighbors_of_v = list(graph.neighbors(v))
    max_value = max(labels[key] for key in neighbors_of_v)
    max_keys = [key for key in neighbors_of_v if labels[key] == max_value]
    return random.choice(max_keys)

def node_to_edge_community(labels,graph):
    communities={}
    for edge in graph.edges():
        u, v = edge
        if labels[u] == labels[v]:
            community=labels[u]
        else : 
            community = random.choice([labels[u],labels[v]])
        if community in communities:
            communities[community].add(edge)
        else : communities[community] ={edge}
    
    comms = [value_set for value_set in communities.values()]
    # print(comms)
    
    return [{tuple(sorted(t)) for t in s} for s in comms]
    # for comm in comms:

def label_propagation(graph):
    stop = False
    labels = {node: node for node in graph.nodes()}
    while not stop:
        stop = True
        shuffled_nodes = list(graph.nodes())
        random.shuffle(shuffled_nodes)
        for v in shuffled_nodes:
            current = labels[v]
            labels[v] = most_frequent_label(labels, graph, v)

            if labels[v] != current:
                stop = False

    communities=node_to_edge_community(labels,graph)
    individual = Individual(communities,graph)
    return individual
    
    


#local expansion
def natural_community(graph, seed):
    nodes = list(graph.neighbors(seed)) + [seed]
    res = graph.subgraph(nodes)
    return res

def local_expansion(graph,given_nodes):
    communities = {}
    visited = {node: 0 for node in given_nodes}
    labels = {node: node for node in graph.nodes()}
    for seed in given_nodes:
        if visited[seed]:
            continue
        visited[seed] = 1
        nat_com = natural_community(graph, seed)
        for node in list(nat_com.nodes()):
            labels[node] = seed
    communities = node_to_edge_community(labels, graph)
    # for i in communities:print(i)
    
    individual = Individual(communities,graph)
    return individual

def local_expansion_random(graph):
    shuffled_nodes = list(graph.nodes())
    random.shuffle(shuffled_nodes)
    return local_expansion(graph,shuffled_nodes)

def local_expansion_by_degree(graph):
    degrees=dict(graph.degree())
    sorted_nodes = sorted(degrees, key = lambda x : degrees[x], reverse = True)
    return local_expansion(graph,sorted_nodes)

def local_expansion_by_eigenvector_centrality(graph):
    eigenvector_centrality = nx.eigenvector_centrality(graph, max_iter=1000)
    sorted_nodes = [node for node, centrality in sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)]
    return local_expansion(graph,sorted_nodes)


# local_expansion_by_degree(graph)
# edges=graph.edges()
# edges=sorted(edges)
# print(edges)

In [3]:
# local_expansion_by_degree(graph)

In [4]:
graph=nx.read_edgelist('graph.txt',nodetype=int)



In [8]:
print(local_expansion_by_eigenvector_centrality(graph))

Loci: [(0, 16), (1, 9), (1, 11), (2, 9), (2, 10), (2, 12), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 20), (2, 21), (2, 22), (2, 26), (3, 4), (3, 6), (3, 10), (3, 19), (3, 24), (3, 25), (3, 28), (4, 5), (4, 6), (4, 7), (4, 27), (4, 29), (4, 30), (5, 8), (5, 13), (5, 23), (6, 31), (12, 32), (17, 33)]
Gene: [(2, 16), (1, 9), (1, 11), (2, 20), (2, 26), (2, 22), (2, 10), (2, 18), (0, 16), (17, 33), (2, 15), (2, 21), (2, 12), (2, 9), (2, 14), (4, 6), (3, 19), (3, 10), (3, 28), (3, 6), (3, 25), (3, 24), (4, 30), (4, 5), (4, 7), (4, 27), (4, 29), (3, 4), (5, 23), (5, 13), (5, 8), (6, 31), (12, 32), (2, 17)]
